In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
class NormalNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        pass